## 导入必要的库

In [ ]:
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter

## 场景 A: 动态控制流 (Define-by-Run)

每次运行，for 循环次数可能不同，导致图结构完全不同

In [ ]:
x = torch.randn(1, requires_grad=True)
y = torch.randn(1, requires_grad=True)

def dynamic_forward(x, y):
    if x.item() > 0:
        for _ in range(2):
            y = y * x
    else:
        y = y + x
    return y

out = dynamic_forward(x, y)
print(f"Current Graph Root: {out.grad_fn}")  # 可能是 MulBackward 或 AddBackward

## 场景 B: 图的销毁

PyTorch 采用"用完即弃"的计算图管理策略

In [ ]:
loss = x * y
loss.backward()  # 第一次反向传播，执行 apply() 并释放中间 Node 内存

try:
    loss.backward()  # 第二次尝试
except RuntimeError as e:
    print(f"\nError captured: {e}")

### 解释

**1. 动态控制流 (Define-by-Run)：**

PyTorch 在每次前向传播时动态构建计算图，因此可以根据输入数据的不同路径执行不同的计算。这使得模型可以灵活地处理各种输入情况，而不需要预定义所有可能的计算路径。

**2. 图的销毁：**

PyTorch 采用"用完即弃"的计算图管理策略。在调用 `backward()` 后，计算图中的中间节点会被释放以节省内存。如果尝试对同一个计算图调用 `backward()`，会因为图已经被销毁而抛出错误。这促使用户在需要多次反向传播时，显式地重新构建计算图。

## 递归遍历计算图

定义一个函数来递归遍历 grad_fn，文本化打印计算图结构

In [ ]:
def print_autograd_graph(node, level=0):
    """
    递归遍历 grad_fn，文本化打印计算图结构
    """
    indent = "    " * level
    # 打印当前节点类型
    print(f"{indent} -> {node}")

    # 检查是否有后续节点
    if hasattr(node, "next_functions"):
        for next_node, _ in node.next_functions:
            # AccumulateGrad 是叶子节点的梯度累加器，通常是终点
            if next_node is not None:
                print_autograd_graph(next_node, level + 1)

### 测试计算图遍历

In [ ]:
x = torch.tensor([2.0], requires_grad=True)
y = torch.tensor([3.0], requires_grad=True)

# 构造一个稍微复杂的路径: z = (x * y) + x
z = x * y
out = z + x

print("=== Autograd Graph Traversal ===")
print(f"Root: {out.grad_fn}")  # AddBackward
print_autograd_graph(out.grad_fn)

### 关于 AccumulateGrad

这种方法最能体现 PyTorch 动态图的本质：图就是一堆链式引用的 C++ 对象。

你可以清晰地看到，因为 x 在计算中参与了两次（一次在乘法里，一次在加法里），所以 AccumulateGrad (对应 x) 在树中出现了两次。这解释了为什么梯度需要累加。

## 使用 TensorBoard 可视化计算图

定义一个简单的网络并使用 TensorBoard 记录图结构

In [ ]:
class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(10, 5)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(5, 1)

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

In [ ]:
model = SimpleNet()
dummy_input = torch.randn(1, 10)

# 使用 SummaryWriter 记录图
writer = SummaryWriter("runs/graph_visualization")

# add_graph 需要模型实例和样例输入，它会追踪一次 forward 过程
writer.add_graph(model, dummy_input)
writer.close()

print("请在终端运行: tensorboard --logdir=runs")